In [1]:
%load_ext autoreload
%autoreload 2

import sys

sys.path.append("./lib/script_parser")
sys.path.append("./lib/bitcoinlib")
sys.path.append("./lib/bitcoincashlib")
sys.path.append(".")


In [2]:
import json
from queue import Queue
from bitcoincash.core.script import CScript
from bitcoincash.core import x
from signatures.utils import buildCTransaction, format_fields
from signatures.extract_signatures import write_signatures
from bitcoincash.core import *
from bitcoincash.core.script import *
from bitcoincash.core.scripteval import (
    VerifyScript,
    SCRIPT_VERIFY_P2SH,
    VerifyScriptError,
)
from utxo_utils.script.script import Script


def extract_sig(raw_tx: dict, input_indices: list, queue: Queue):
    format_fields(raw_tx)
    tx = buildCTransaction(raw_tx, queue)

    prevouts = {
        i: CScript(x(raw_tx["inputs"][i]["spent_script_hex"]))
        for i in range(len(tx.vin))
    }
    prevvalues = {i: raw_tx["inputs"][i]["value"] for i in range(len(tx.vin))}

    flags = {SCRIPT_VERIFY_P2SH}
    ins = SCRIPT_VERIFY_P2SH in flags
    for i in input_indices:
        try:
            VerifyScript(
                tx.vin[i].scriptSig,
                prevouts[i],
                tx,
                i,
                flags=flags,
                amount=prevvalues[i],
            )
        except VerifyScriptError as e:
            pass
        except Exception as e:
            redeem_script = raw_tx["inputs"][i]["script_asm"].split(" ")[-1]
            parsed_redeem_script = Script(redeem_script, chain="bch").decoded
            """print(
                f"Error {e} for tx ",
                raw_tx["transaction_hash"],
                " at index ",
                i,
                redeem_script,
            )"""


def collect_sig(queue):
    subset = {}

    with open(
        "/Users/vincent/Documents/PhD/Blockchains/UTXO/ecdsa-signatures/local/data/bch/bch_tx.json"
    ) as f:
        for n, line in enumerate(f):
            raw_tx = json.loads(line)
            tx_hash = raw_tx["transaction_hash"]
            if not subset or tx_hash in subset:
                indices = (
                    range(len(raw_tx["inputs"]))
                    if not subset or not subset[tx_hash]
                    else subset[tx_hash]
                )
                extract_sig(raw_tx, indices, queue)


queue = Queue()
collect_sig(queue)
queue.put("")
write_signatures(
    "/Users/vincent/Documents/PhD/Blockchains/UTXO/ecdsa-signatures/local/signatures/bch",
    queue,
    10**6,
)